# 🖊️ SignVerifAI - FAR/EER Optimization (v3.0)

**New Features:**
- 🎯 **Dual LR**: backbone_lr=1e-5, head_lr=3e-4
- 🔥 **Hybrid Loss**: 0.5×Contrastive + 0.5×Triplet (batch-hard)
- 🧊 **Freeze/Unfreeze**: 3 epoch backbone freeze
- 🛡️ **Security Threshold**: FAR@FRR=1%
- ⚡ **Mixed Precision (AMP)**

**Target Metrics:**
- AUC >= 0.95
- EER <= 0.08
- FAR@FRR=1% <= 0.10

---

## 1️⃣ GPU Check & Setup

In [ ]:
!nvidia-smi

import torch
print(f"\nPyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

## 2️⃣ Clone & Install

In [ ]:
!rm -rf SignVerifAI
!git clone https://github.com/gorkemelih/SignVerifAI.git
%cd SignVerifAI
!pip install -e . -q
print("✅ Kurulum tamamlandı!")

## 3️⃣ Mount Google Drive & Load Data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
DRIVE_DATA_PATH = "/content/drive/MyDrive/SignVerifAI/data_processed.zip"

if os.path.exists(DRIVE_DATA_PATH):
    !unzip -q -o {DRIVE_DATA_PATH} -d .
    print("✅ Veriler yüklendi!")
else:
    print(f"❌ Dosya bulunamadı: {DRIVE_DATA_PATH}")

## 4️⃣ Prepare Data Splits & Pairs

In [ ]:
import pandas as pd
metadata = pd.read_csv('data_processed/metadata.csv')
print(f"Toplam görüntü: {len(metadata)}")
print(f"Kişi sayısı: {metadata['person_id'].nunique()}")
print(f"\nEtiket dağılımı:")
print(metadata['label'].value_counts())

In [ ]:
!signverify split
!signverify pairs

## 5️⃣ Train Model 🚀

**Key Optimizations:**
- Hybrid Loss (Contrastive + Triplet)
- Dual LR: backbone=1e-5, head=3e-4
- Freeze backbone for 3 epochs
- Hard negative mining (50%)
- OneCycleLR scheduler

In [ ]:
# Training with optimized parameters
!signverify train \
    --device cuda \
    --epochs 40 \
    --batch-size 128 \
    --freeze-epochs 3 \
    --backbone-lr 1e-5 \
    --head-lr 3e-4 \
    --loss hybrid \
    --hard-mining \
    --scheduler onecycle \
    --log-every 5

## 6️⃣ Evaluate Model with Security Threshold

In [ ]:
!signverify eval --device cuda --threshold-mode all

In [ ]:
from IPython.display import Image, display, Markdown

print("📊 ROC Curve:")
display(Image('outputs/reports/roc_curve.png'))

print("\n📊 Score Distribution:")
display(Image('outputs/reports/score_distribution.png'))

print("\n📊 Confusion Matrix:")
display(Image('outputs/reports/confusion_matrix.png'))

In [ ]:
with open('outputs/reports/eval_report.md', 'r') as f:
    display(Markdown(f.read()))

In [ ]:
# Training history
import pandas as pd
import matplotlib.pyplot as plt

history = pd.read_csv('outputs/reports/train_history.csv')

fig, axes = plt.subplots(2, 2, figsize=(12, 8))

axes[0,0].plot(history['epoch'], history['train_loss'], label='Train')
axes[0,0].plot(history['epoch'], history['val_loss'], label='Val')
axes[0,0].set_title('Loss')
axes[0,0].legend()

axes[0,1].plot(history['epoch'], history['auc'])
axes[0,1].axhline(y=0.95, color='r', linestyle='--', label='Target')
axes[0,1].set_title('AUC')
axes[0,1].legend()

axes[1,0].plot(history['epoch'], history['eer'])
axes[1,0].axhline(y=0.08, color='r', linestyle='--', label='Target')
axes[1,0].set_title('EER')
axes[1,0].legend()

axes[1,1].plot(history['epoch'], history['lr'])
axes[1,1].set_title('Learning Rate')

plt.tight_layout()
plt.show()

## 7️⃣ Save Model to Drive

In [ ]:
import shutil
import glob

runs = sorted(glob.glob('outputs/models/run_*'))
if runs:
    latest_run = runs[-1]
    dest = "/content/drive/MyDrive/SignVerifAI/trained_models/"
    os.makedirs(dest, exist_ok=True)
    
    shutil.copy(f"{latest_run}/checkpoint_best.pt", dest)
    shutil.copy(f"{latest_run}/config.json", dest)
    shutil.copy(f"{latest_run}/train_history.csv", dest)
    
    reports_dest = "/content/drive/MyDrive/SignVerifAI/reports/"
    os.makedirs(reports_dest, exist_ok=True)
    for f in glob.glob('outputs/reports/*'):
        shutil.copy(f, reports_dest)
    
    print(f"✅ Model kaydedildi: {dest}")
    print(f"✅ Raporlar kaydedildi: {reports_dest}")
else:
    print("❌ Eğitilmiş model bulunamadı.")

---
## 📋 v3.0 Configuration Summary

| Setting | Value |
|---------|-------|
| Epochs | 40 |
| Batch Size | 128 |
| Freeze Epochs | 3 |
| Backbone LR | 1e-5 |
| Head LR | 3e-4 |
| Loss | Hybrid (Contrastive + Triplet) |
| Scheduler | OneCycleLR |
| Hard Mining | 50% |
| Threshold Modes | EER, Accuracy, F1, Security |